In [18]:
#Getting started
from __future__ import print_function
import torch

x = torch.Tensor(5, 3)
print(x)

x = torch.rand(5, 3)
print(x)

print(x.size())

y = torch.rand(5, 3)
print(x + y)

print(torch.add(x, y))

result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)

y.add_(x)
print(y)

print(x[:, 1])

a = torch.ones(5)
print(a)

b = a.numpy()
print(b)

a.add_(1)
print(a)
print(b)

import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y


1.00000e-14 *
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  8.4713  0.0000  0.0000
  0.0000  0.0000  0.0000
  8.4722  0.0000  0.0000
[torch.FloatTensor of size 5x3]


 0.3169  0.3705  0.9052
 0.6914  0.9115  0.5938
 0.1342  0.4113  0.2462
 0.8131  0.8142  0.7780
 0.4836  0.2528  0.0014
[torch.FloatTensor of size 5x3]

torch.Size([5, 3])

 1.1761  0.5192  1.2383
 0.9282  1.4091  0.7019
 0.3134  0.7541  0.3518
 1.0931  1.5559  1.0150
 0.8422  0.5397  0.5479
[torch.FloatTensor of size 5x3]


 1.1761  0.5192  1.2383
 0.9282  1.4091  0.7019
 0.3134  0.7541  0.3518
 1.0931  1.5559  1.0150
 0.8422  0.5397  0.5479
[torch.FloatTensor of size 5x3]


 1.1761  0.5192  1.2383
 0.9282  1.4091  0.7019
 0.3134  0.7541  0.3518
 1.0931  1.5559  1.0150
 0.8422  0.5397  0.5479
[torch.FloatTensor of size 5x3]


 1.1761  0.5192  1.2383
 0.9282  1.4091  0.7019
 0.3134  0.7541  0.3518
 1.0931  1.5559  1.0150
 0.8422  0.5397  0.5479
[torch.FloatTensor of size 5x3]


 0.3705
 0.9115
 0.4113
 0.8142
 0.25

In [20]:
#AutoGrad
import torch
from torch.autograd import Variable

x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

y = x + 2
print(y)

print(y.grad_fn)

z = y * y * 3
out = z.mean()

print(z, out)

out.backward()

print(x.grad)

x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

Variable containing:
-352.8688
 930.4115
 660.7255
[torch.FloatTensor of size 3]

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



In [26]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

params = list(net.parameters())
print(len(params))
print(params[0].size())

input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

net.zero_grad()
out.backward(torch.randn(1, 10))

output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

net.zero_grad() 

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() 

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)
10
torch.Size([6, 1, 5, 5])
Variable containing:
1.00000e-02 *
  1.6548  8.9959 -0.4060  3.2363 -8.3231  1.1483 -9.3527  8.6833  9.2969 -1.1513
[torch.FloatTensor of size 1x10]

Variable containing:
 38.3585
[torch.FloatTensor of size 1]

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  7.9970
 -6.7365
 -0.1598
 -1.8182
  2.3142
 -0.7646
[torch.FloatTensor of size 6]



In [28]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


import matplotlib.pyplot as plt
import numpy as np



def imshow(img):
    img = img / 2 + 0.5    
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))



dataiter = iter(trainloader)
images, labels = dataiter.next()


imshow(torchvision.utils.make_grid(images))

print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data


        inputs, labels = Variable(inputs), Variable(labels)


        optimizer.zero_grad()


        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.data[0]
        if i % 2000 == 1999:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

outputs = net(Variable(images))

_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
    
net.cuda()

inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

Files already downloaded and verified
Files already downloaded and verified
  cat plane truck plane
[1,  2000] loss: 2.177
[1,  4000] loss: 1.863
[1,  6000] loss: 1.680
[1,  8000] loss: 1.577
[1, 10000] loss: 1.519
[1, 12000] loss: 1.495
[2,  2000] loss: 1.428
[2,  4000] loss: 1.366
[2,  6000] loss: 1.361
[2,  8000] loss: 1.350
[2, 10000] loss: 1.311
[2, 12000] loss: 1.296
Finished Training
GroundTruth:    cat  ship  ship plane
Predicted:    cat  ship   car  ship
Accuracy of the network on the 10000 test images: 55 %
Accuracy of plane : 50 %
Accuracy of   car : 68 %
Accuracy of  bird : 34 %
Accuracy of   cat : 34 %
Accuracy of  deer : 47 %
Accuracy of   dog : 56 %
Accuracy of  frog : 64 %
Accuracy of horse : 55 %
Accuracy of  ship : 76 %
Accuracy of truck : 63 %


RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at D:\Projects\pytorch\torch\lib\THC\THCGeneral.c:70